In [4]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('build/simulation_example_1/stats/stats_simd_best.csv',header=0)
print(df)


df.plot(x='Generation', y = 'fitness')

TypeError: Passing a bool to header is invalid. Use header=None for no header or header=int or list-like of ints to specify the row(s) making up the column names